In [2]:
import numpy as np
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
data = open('./data/input.txt','r')
data = list(data)
data_use = []
data_f = []
index = 0
temp = ''
for d in data:
    if d == "<start>\r\n":
        index = 1
        continue
    if d == '<end>\r\n':
        index = 0
    if index == 1:
        temp = temp + str(d)
    if index == 0:
        data_use.append(temp)
        temp = ''
#print(d)
for d in data_use:
    if len(d) > 40:
        data_f.append(d)
data_use = data_f
all_character = set()
counter = 0
for d in data_use:
    for dd in d:
        all_character.add(dd)
all_character = list(all_character)
nletter = len(all_character)

In [4]:
print(data_use[2])

XIV?me si?cle)
Z:Transcrit et/ou corrig? par Michel BELLON - 2005-03-27
Z:Pour toute observation mailto:galouvielle@free.fr
M:2/2
L:1/4
Q:1/4=110
K:Fmaj
V:1
dd/2c/2 dB| GB c2|_e/2d/2B/2c/2 AG|Bc c/2B/2 G|F4|dd/2c/2 dB| GB c2|_e/2d/2B/2c/2 AG|Bc c/2B/2 G/2A/4F/4|G2 zG|
w:Stel-__la splen-dens in mon-____te ut so-lis ra-_di-um Mi-__ra-cu-lis ser-ra-____to ex-au-di po-_pu-__lum  Prin-
Gd eg|ef/2d/2 ce|fd c/2B/2B/2A/2|G2 zG|Gd eg|ef/2d/2 ce|fd c/2B/2B/2A/2|G2 z2||
w:ci-pes et ma-gna-__ tes ex-stir-pe re-_gi-_a sae-cu-li po-tes-ta-__tes ob-ten-ta ve-_ni-_a
dd/2c/2 dB| GB c2|_e/2d/2B/2c/2 AG|Bc c/2B/2 G|F4|dd/2c/2 dB| GB c2|_e/2d/2B/2c/2 AG|Bc c/2B/2 G/2A/4F/4|G4|]
w:Pe-__cca mi num pro-cla-____mant tu den tes pec-_to-ra Po-__pli te fle-xo cla-____mant hic A-ve Ma-_ri-__a
V:2
G2 GB|dG F2|B2 cd|GF cd|f4| G2 GB|dG F2|B2 cd|GF Bc|d2 zd-|
d G2 G|cB A2|FG Bc|d2 zd-|d G2 G|cB A2|FG Bc|d2 z2|
G2 GB|dG F2|B2 cd|GF cd|f4| G2 GB|dG F2|B2 cd|GF Bc|d4|]



In [5]:
def lineToTensor(line):
    tensor = torch.zeros(len(line),nletter)
    position = np.zeros(len(line))
    for i, letter in enumerate(line):
        position[i] = all_character.index(letter)
        tensor[i][all_character.index(letter)] = 1
    position = torch.from_numpy(position[1:]).long()
    return autograd.Variable(tensor[:-1,:]).cuda(),autograd.Variable(position).cuda()

In [6]:
def batch_g(data,batch_size):
    lyric = np.random.randint(len(data))
    #print(len(data[lyric]))
    if len(data[lyric])-batch_size >0:
        interval_s = np.random.randint(len(data[lyric])-batch_size)
        return data[lyric][interval_s:interval_s+batch_size]
    else:
        return []

In [7]:
class LSTM(nn.Module):
    def __init__(self,letter_dim,hidden_dim):
        super(LSTM, self).__init__()
        self.letter_dim = letter_dim
        self.hidden_dim = hidden_dim
        self.hidden = self.init_hidden()
        
        self.lstm = nn.LSTM(letter_dim,hidden_dim)
        self.hidden2out = nn.Linear(hidden_dim,letter_dim)
    def init_hidden(self):
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)).cuda(),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)).cuda())
    def forward(self,line):
        lstm_out, self.hidden = self.lstm(line.view(len(line),1,-1), self.hidden)
        out_space = self.hidden2out(lstm_out.view(len(lstm_out),-1))
#         out_scores = F.softmax(out_space / 1.0,dim = 1)
        return out_space

In [8]:
hidden_dim = 100
batch_size = 30
lstm = LSTM(nletter,hidden_dim).cuda()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm.parameters(), lr=0.001)

In [9]:
for epoch in range(1000):
    lstm.zero_grad()
    batch = batch_g(data_use,batch_size)
#     print(batch)
#     print('\n')
    if len(batch)>0:
        batch_in,batch_t = lineToTensor(batch)
        lstm.hidden = lstm.init_hidden()
        out_scores = lstm(batch_in)
        loss = loss_function(out_scores,batch_t)
        if epoch % 100 == 0:
            print(loss)
        loss.backward()
        optimizer.step()

Variable containing:
 4.5674
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 3.4689
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 3.1698
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 3.7864
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 3.4241
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 3.6139
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 4.6997
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 2.6456
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 2.4500
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 2.6399
[torch.cuda.FloatTensor of size 1 (GPU 0)]



In [13]:
print(F.softmax(out_scores / 1.0,dim = 1).data.cpu().numpy()[0])

[ 0.05511661  0.01118735  0.01267653  0.00664901  0.01130545  0.0072906
  0.0027741   0.00361837  0.0171743   0.00527951  0.00740505  0.00666164
  0.00825582  0.00080569  0.00475316  0.02872356  0.01045729  0.01165287
  0.00686893  0.00936833  0.00304742  0.02907304  0.00447163  0.00694241
  0.00159121  0.02426892  0.01985937  0.00530077  0.00423735  0.00912572
  0.02659946  0.00693125  0.00485062  0.00538935  0.00330997  0.00351124
  0.00273532  0.02539418  0.02057483  0.00468945  0.01311214  0.0097442
  0.00462107  0.00226491  0.0134381   0.00907961  0.00242877  0.00076565
  0.00679352  0.0485329   0.00690124  0.03684217  0.01105658  0.03172969
  0.01702657  0.00361986  0.00433531  0.0063018   0.00322848  0.00602502
  0.00773805  0.00858571  0.02482934  0.00457505  0.01031361  0.01242657
  0.00594076  0.00632592  0.00991871  0.00066337  0.02400466  0.00390882
  0.00786336  0.00949539  0.0115975   0.00470597  0.0044308   0.00707852
  0.02674899  0.01646118  0.00258536  0.00762452  0.0

In [19]:
present = torch.zeros(1,nletter)
present[0][all_character.index('X')] = 1
present = autograd.Variable(present).cuda()
lyric = []
lyric.append('X')
for num_generation in range(400):
    p = F.softmax(lstm(present)/1.0,dim=1).data.cpu().numpy().reshape(-1)
    next_char_index = int(np.random.choice(nletter,1,p=p))
    next_char = all_character[next_char_index]
    lyric.append(next_char)
    present = torch.zeros(1,nletter)
    present[0][all_character.index(next_char)] = 1
    present = autograd.Variable(present).cuda()

In [20]:
print int(next_char_index)

87


In [21]:
print(str(lyric))

['X', 'A', 'A', 'B', '2', 'B', '2', ' ', ' ', 'D', '(', 'G', '2', 'c', ' ', 'F', '|', 'd', '2', ' ', '\r', 'G', 'E', ' ', '|', ' ', 'F', '3', 'B', 'F', 'K', 'D', ' ', ' ', '|', '0', ' ', ' ', 'F', ' ', '|', ' ', 'D', '2', 'd', 'E', '2', 'd', 'd', 'G', 'e', 'c', ' ', 'B', 'A', ' ', '|', 'G', '(', '|', 'B', ' ', 'D', ' ', 'c', 'G', '2', 'G', 'B', '\r', 'A', 'F', '/', 'G', '~', '2', 'G', 'A', ':', ' ', '3', 'c', '2', 'f', ' ', 'G', '/', '2', '/', 'A', 'G', 'G', ' ', 'd', '2', 'G', '2', ' ', 'G', ' ', 'A', 'B', '|', 'd', ' ', '2', 'g', 'd', ' ', 'd', 'c', ' ', '|', 'G', '|', 'e', 'A', 'D', 'c', 'D', ' ', 'E', ' ', 'B', '2', ' ', 'B', '|', '2', '!', 'F', ':', 'd', 'A', '|', 'G', 'A', 'G', ' ', 'c', '3', ' ', 'F', 'c', 'A', ' ', 'A', 'D', '|', ' ', 'G', '2', 'G', '|', 'F', 'c', 'f', 'G', '2', 'A', 'd', '2', 'c', 'c', 'G', 'g', '2', 'G', 'B', '|', 'G', ' ', '|', 'D', 'T', '|', ' ', '|', ' ', 'A', '|', 'A', '/', 'A', '2', 'd', 'B', '|', 'A', 'A', '(', 'f', ' ', ' ', 'c', 'f', ' ', 'd', '2', 'A

In [22]:
Music = ""
for char in lyric:
    Music = Music + char
print(Music)

AF/G~2GA: 3c2f G/2/AGG d2G2 G AB|d 2gd dc |G|eADcD E B2 B|2!F:dA|GAG c3 FcA AD| G2G|FcfG2Ad2ccGg2GB|G |DT| | A|A/A2dB|AA(f  cf d2A|  GBiG2 ed22  BDFA|| B2  EA  GB/E2  |GF||S dc2  
BMD"g| dg| ^cBA2|dB|G f2 g22 B, |2a d  | AAG Gd e2  22ceA dB Ge
